In [1]:
import pandas as pd
import numpy as np

In [4]:
test_data = pd.read_csv("../../tests/toy_data_large.csv")

In [6]:
test_data.head()

,visitorId,visitNumber,transactionId,transactionRevenue,startDate,endDate,campaignId
0,7827,4,NaN,NaN,2023/7/8 10:18,2023/11/15 13:31,100
1,6604,1,NaN,NaN,2023/10/9 17:33,2023/11/18 07:03,102
2,7509,2,NaN,NaN,2023/11/26 05:34,2023/12/10 20:44,103
3,8874,1,NaN,NaN,2023/5/27 10:25,2023/12/23 01:45,102
4,9417,1,NaN,NaN,2023/1/23 17:15,2023/5/10 00:27,104


In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visitorId           500 non-null    int64  
 1   visitNumber         500 non-null    int64  
 2   transactionId       104 non-null    float64
 3   transactionRevenue  104 non-null    float64
 4   startDate           500 non-null    object 
 5   endDate             500 non-null    object 
 6   campaignId          500 non-null    int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 27.5+ KB


In [14]:
print('the beginning date of the start date: {}'.format(test_data['startDate'].min()))
print('the last date of the start date: {}'.format(test_data['startDate'].max()))

the beginning date of the start date: 2023/1/1 20:29
the last date of the start date: 2023/9/9 03:54


In [15]:
print('the beginning date of the end date: {}'.format(test_data['endDate'].min()))
print('the last date of the end date: {}'.format(test_data['endDate'].max()))

the beginning date of the end date: 2023/1/21 14:59
the last date of the end date: 2023/9/9 17:07


In [19]:
def compute_metrics(data, campaign_id, start_date, end_date):
    
    data['startDate'] = pd.to_datetime(data['startDate'])
    data['endDate'] = pd.to_datetime(data['endDate'])

    filtered_data = data[(data['campaignId'] == campaign_id) & 
                         (data['startDate'] >= start_date) & 
                         (data['endDate'] <= end_date)]

    new_visitors = filtered_data['visitNumber'] == 1
    returning_visitors = filtered_data['visitNumber'] > 1
    new_to_return_rate = new_visitors.sum() / returning_visitors.sum() if returning_visitors.sum() > 0 else 0

    conversions = filtered_data['transactionId'].notna()
    conversion_rate = conversions.sum() / len(filtered_data) if len(filtered_data) > 0 else 0

    total_transaction_revenue = filtered_data['transactionRevenue'].sum()
    average_transaction_revenue = filtered_data['transactionRevenue'][conversions].mean() if conversions.sum() > 0 else 0

    return {
        'new_to_return_rate': new_to_return_rate,
        'conversion_rate': conversion_rate,
        'total_transaction_revenue': total_transaction_revenue,
        'average_transaction_revenue': average_transaction_revenue
    }


example_campaign_id = 100
example_start_date = pd.to_datetime('2023-01-01')
example_end_date = pd.to_datetime('2023-12-31')

metrics = compute_metrics(test_data, example_campaign_id, example_start_date, example_end_date)
metrics


{'new_to_return_rate': 1.1666666666666667,
 'conversion_rate': 0.15384615384615385,
 'total_transaction_revenue': 4718.21007335,
 'average_transaction_revenue': 262.1227818527778}

In [21]:
import pandas as pd
import numpy as np


def compute_metrics(data, campaign_id, start_date, end_date):
    data['startDate'] = pd.to_datetime(data['startDate'])
    data['endDate'] = pd.to_datetime(data['endDate'])

    filtered_data = data[(data['campaignId'] == campaign_id) & 
                         (data['startDate'] >= start_date) & 
                         (data['endDate'] <= end_date)]

    new_visitors = filtered_data['visitNumber'] == 1
    returning_visitors = filtered_data['visitNumber'] > 1
    new_to_return_rate = new_visitors.sum() / returning_visitors.sum() if returning_visitors.sum() > 0 else 0

    conversions = filtered_data['transactionId'].notna()
    conversion_rate = conversions.sum() / len(filtered_data) if len(filtered_data) > 0 else 0

    total_transaction_revenue = filtered_data['transactionRevenue'].sum()
    average_transaction_revenue = filtered_data['transactionRevenue'][conversions].mean() if conversions.sum() > 0 else 0

    return {
        'new_to_return_rate': new_to_return_rate,
        'conversion_rate': conversion_rate,
        'total_transaction_revenue': total_transaction_revenue,
        'average_transaction_revenue': average_transaction_revenue
    }


def find_campaigns(data, start_date, end_date, campaign_ids, metric):
    campaign_metrics = {}
    for cid in campaign_ids:
        metrics = compute_metrics(data, cid, start_date, end_date)
        campaign_metrics[cid] = metrics.get(metric, 0)

    best_campaign = max(campaign_metrics, key=campaign_metrics.get)
    worst_campaign = min(campaign_metrics, key=campaign_metrics.get)

    return {
        'best_campaign': {'id': best_campaign, 'value': campaign_metrics[best_campaign]},
        'worst_campaign': {'id': worst_campaign, 'value': campaign_metrics[worst_campaign]}
    }



example_campaign_ids = [100, 101, 102, 103, 104]
example_metric = 'conversion_rate'
example_start_date = pd.to_datetime('2023-01-01')
example_end_date = pd.to_datetime('2023-12-31')


best_and_worst_campaigns = find_campaigns(test_data, example_start_date, example_end_date, example_campaign_ids, example_metric)
best_and_worst_campaigns



{'best_campaign': {'id': 104, 'value': 0.25},
 'worst_campaign': {'id': 100, 'value': 0.15384615384615385}}